In [1]:
import pandas as pd
import os

Para poder seleccionar bien las variables a utilizar, creo un dataframe mucho mas chico para no tener que procesar la totalidad de los datos.

In [2]:
df = pd.read_parquet('../data/datos-todas-estaciones.parquet')
df.columns = [col.lower().strip() for col in df.columns]

df_mini = df.copy()[:1000]

### Selección de variables.

In [11]:
df.sample(10)

,fecha,id_estacion,temperatura_abrigo_150cm_minima,temperatura_abrigo_150cm_maxima,temperatura_abrigo_150cm,precipitacion_pluviometrica,humedad_media_8_14_20,tesion_vapor_media,rocio_medio,horas_frio
897294,1986-07-24 00:00:00.0,NH0502,3.2,21.4,12.300000,0.00000,75.0,10.400000,7.500000,5.010989
314100,2022-11-07 00:00:00.0,A872924,8.2,23.4,16.202780,0.00000,37.0,7.799369,3.387188,0.000000
270436,2019-02-08 00:00:00.0,A872909,20.9,38.2,29.183340,0.00000,66.0,20.926620,NaN,0.000000
266145,2019-06-15 00:00:00.0,A872908,21.1,29.4,24.189580,0.00000,70.0,21.489460,18.641640,0.000000
59313,2014-04-10 00:00:00.0,A872819,8.5,19.8,13.552780,0.00000,74.0,11.635050,9.123349,0.000000
194575,2015-12-10 00:00:00.0,A872867,15.6,32.0,23.615280,0.00000,64.0,19.927240,17.347800,0.000000
180332,2013-11-09 00:00:00.0,A872862,17.9,24.8,19.788890,46.99998,89.0,21.221980,18.441050,0.000000
40358,2023-04-29 00:00:00.0,A872812,17.2,33.1,23.867360,0.00000,71.0,24.393930,20.552640,0.000000
792168,2022-09-16 00:00:00.0,NH0460,9.0,25.0,17.000000,0.00000,60.0,13.348049,11.060313,NaN
219749,2015-06-28 00:00:00.0,A872885,2.8,19.3,9.321527,0.00000,76.0,8.595918,4.758533,10.458000


In [3]:
features = (df.isna().sum() * 100 / len(df)).reset_index()
features.columns = ['columna', 'percentNa']

features.sort_values('percentNa').query('percentNa <= 99')

,columna,percentNa
0,fecha,0.000000
30,id_estacion,0.000000
3,temperatura_abrigo_150cm_minima,6.066566
2,temperatura_abrigo_150cm_maxima,6.360656
1,temperatura_abrigo_150cm,6.553649
11,precipitacion_pluviometrica,7.020599
18,humedad_media_8_14_20,9.662863
16,tesion_vapor_media,10.929604
19,rocio_medio,12.666109
28,horas_frio,14.436710


In [4]:
cols = features.sort_values('percentNa').query('percentNa <= 15').columna.values
cols

array(['fecha', 'id_estacion', 'temperatura_abrigo_150cm_minima',
       'temperatura_abrigo_150cm_maxima', 'temperatura_abrigo_150cm',
       'precipitacion_pluviometrica', 'humedad_media_8_14_20',
       'tesion_vapor_media', 'rocio_medio', 'horas_frio'], dtype=object)

In [5]:
unidades = {'fecha': 'fecha',
 'temperatura_abrigo_150cm': '°C',
 'temperatura_abrigo_150cm_maxima': '°C',
 'temperatura_abrigo_150cm_minima': '°C',
 'temperatura_intemperie_5cm_minima': '°C',
 'temperatura_intemperie_50cm_minima': '°C',
 'temperatura_suelo_5cm_media': '°C',
 'temperatura_suelo_10cm_media': '°C',
 'temperatura_inte_5cm': '°C',
 'temperatura_intemperie_150cm_minima': '°C',
 'humedad_suelo': '%',
 'precipitacion_pluviometrica': 'mm',
 'precipitacion_cronologica': 'mm',
 'precipitacion_maxima_30minutos': 'mm',
 'heliofania_efectiva': 'horas',
 'heliofania_relativa': '%',
 'tesion_vapor_media': 'hPa',
 'humedad_media': '%',
 'humedad_media_8_14_20': '%',
 'rocio_medio': '°C',
 'duracion_follaje_mojado': 'horas',
 'velocidad_viento_200cm_media': 'm/s',
 'direccion_viento_200cm': 'grados',
 'velocidad_viento_1000cm_media': 'm/s',
 'direccion_viento_1000cm': 'grados',
 'velocidad_viento_maxima': 'm/s',
 'presion_media': 'hPa',
 'radiacion_global': 'MJ/m²',
 'horas_frio': 'horas',
 'unidades_frio': 'unidades'}

unidades_completas = {
    "fecha": "Fecha (día, mes, año, hora)",
    "°C": "Grados Celsius (temperatura)",
    "%": "Porcentaje (%)",
    "mm": "Milímetros (precipitación)",
    "horas": "Horas",
    "hPa": "Hectopascales (presión o tensión de vapor)",
    "m/s": "Metros por segundo (velocidad del viento)",
    "grados": "Grados (dirección del viento, 0° a 360°)",
    "MJ/m²": "Megajulios por metro cuadrado (radiación solar)",
    "unidades": "Unidades de frío (índice agroclimático)"
}


## Valores nulos.

In [6]:
pd.set_option("display.max_columns", 200)

In [7]:
df = df[cols]
df.isna().sum()

fecha                                   0
id_estacion                             0
temperatura_abrigo_150cm_minima     56047
temperatura_abrigo_150cm_maxima     58764
temperatura_abrigo_150cm            60547
precipitacion_pluviometrica         64861
humedad_media_8_14_20               89272
tesion_vapor_media                 100975
rocio_medio                        117018
horas_frio                         133376
dtype: int64

In [8]:
df.isna().sum()

fecha                                   0
id_estacion                             0
temperatura_abrigo_150cm_minima     56047
temperatura_abrigo_150cm_maxima     58764
temperatura_abrigo_150cm            60547
precipitacion_pluviometrica         64861
humedad_media_8_14_20               89272
tesion_vapor_media                 100975
rocio_medio                        117018
horas_frio                         133376
dtype: int64

Detectar las filas en donde la temperatura minima o maxima sean nulos pero la "temperatura_abrigo_150cm" tenga algun valor, para poder incrustar ese valor en la minima y la maxima.

In [21]:
df.loc[
    ((df.temperatura_abrigo_150cm_minima.isna()) | (df.temperatura_abrigo_150cm_maxima.isna()))& (~df.temperatura_abrigo_150cm.isna())
]

,fecha,id_estacion,temperatura_abrigo_150cm_minima,temperatura_abrigo_150cm_maxima,temperatura_abrigo_150cm,precipitacion_pluviometrica,humedad_media_8_14_20,tesion_vapor_media,rocio_medio,horas_frio
31,2009-09-06 00:00:00.0,A872801,NaN,NaN,17.01880,NaN,71.0,14.43417,12.425080,0.0
46,2009-09-21 00:00:00.0,A872801,NaN,NaN,17.26667,NaN,51.0,11.16542,8.536985,0.0
56,2009-10-05 00:00:00.0,A872801,NaN,NaN,21.57314,NaN,79.0,18.58017,16.252500,0.0
157,2010-08-27 00:00:00.0,A872801,NaN,NaN,14.47664,NaN,83.0,14.33899,12.238580,0.0
515,2012-03-16 00:00:00.0,A872801,NaN,NaN,17.99207,NaN,NaN,16.23893,14.219770,0.0
...,...,...,...,...,...,...,...,...,...,...
863604,1995-04-30 00:00:00.0,NH0496,NaN,NaN,13.15000,2.5,NaN,NaN,NaN,0.0
919081,2012-08-03 00:00:00.0,NH0550,NaN,NaN,8.40000,0.0,65.0,7.20000,2.200000,0.0
919689,2014-04-03 00:00:00.0,NH0550,NaN,NaN,25.80000,6.0,89.0,29.50000,23.800000,0.0
919710,2014-04-24 00:00:00.0,NH0550,NaN,NaN,20.75000,0.0,77.0,18.80000,16.600000,0.0


Cambio los valores nulos de la fila de horas_frio por 0.0

In [20]:
df.horas_frio = df.horas_frio.fillna(0.0)

Para los datos de **temperatura_abrigo_150cm_maxima** y **temperatura_abrigo_150cm_maxima** que tiene  valores nulos pero la columna **temperatura_abrigo_150cm** tiene algun valor, se me ocrrio de hacer lo siguiente:

Detectar en el mes de ese registro cuanto difieren los valores de minima y maxima con el de **temperatura_abrigo_150cm**. Despues sacar un promedio y el resultado asignarselo a los valores nulos.

Si hay una semana con valores nulos en la temperatura, imputar los valores de la misma  pero de un año anterior. 

In [23]:
(df.temperatura_abrigo_150cm_minima - df.temperatura_abrigo_150cm).mean()

np.float64(-6.47521352250143)

In [25]:
df[
    (df.temperatura_abrigo_150cm_minima.isna()) & (df.temperatura_abrigo_150cm_minima.shift(1).notna())
]

,fecha,id_estacion,temperatura_abrigo_150cm_minima,temperatura_abrigo_150cm_maxima,temperatura_abrigo_150cm,precipitacion_pluviometrica,humedad_media_8_14_20,tesion_vapor_media,rocio_medio,horas_frio
20,2009-08-26 00:00:00.0,A872801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
30,2009-09-05 00:00:00.0,A872801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
43,2009-09-18 00:00:00.0,A872801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
46,2009-09-21 00:00:00.0,A872801,NaN,NaN,17.26667,NaN,51.0,11.165420,8.536985,0.0
56,2009-10-05 00:00:00.0,A872801,NaN,NaN,21.57314,NaN,79.0,18.580170,16.252500,0.0
...,...,...,...,...,...,...,...,...,...,...
923792,2025-06-29 00:00:00.0,NH0550,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
923802,2025-07-09 00:00:00.0,NH0550,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
923818,2025-07-25 00:00:00.0,NH0550,NaN,16.5,NaN,3.0,88.0,14.304822,12.277325,0.0
923826,2025-08-02 00:00:00.0,NH0550,NaN,20.5,NaN,0.0,NaN,NaN,NaN,0.0


In [26]:
df[
    (df.temperatura_abrigo_150cm_minima.notna()) & (df.temperatura_abrigo_150cm_minima.shift(-1).isna())
]

,fecha,id_estacion,temperatura_abrigo_150cm_minima,temperatura_abrigo_150cm_maxima,temperatura_abrigo_150cm,precipitacion_pluviometrica,humedad_media_8_14_20,tesion_vapor_media,rocio_medio,horas_frio
19,2009-08-25 00:00:00.0,A872801,12.1,27.6,18.62847,0.0,65.0,15.120320,13.128280,0.000000
29,2009-09-04 00:00:00.0,A872801,9.7,16.8,12.92362,NaN,93.0,13.497310,11.394110,0.000000
42,2009-09-17 00:00:00.0,A872801,10.8,14.9,12.77292,NaN,90.0,13.144750,10.889290,0.000000
45,2009-09-20 00:00:00.0,A872801,3.7,22.4,12.98750,0.0,59.0,9.660729,6.431308,6.473997
55,2009-10-04 00:00:00.0,A872801,13.4,23.7,18.44653,29.6,66.0,15.116120,13.101610,0.000000
...,...,...,...,...,...,...,...,...,...,...
923801,2025-07-08 00:00:00.0,NH0550,10.4,28.4,19.40000,0.0,NaN,NaN,NaN,0.000000
923817,2025-07-24 00:00:00.0,NH0550,8.5,11.2,9.85000,0.0,NaN,NaN,NaN,0.000000
923825,2025-08-01 00:00:00.0,NH0550,12.5,28.4,20.45000,0.0,76.0,17.947170,15.665318,0.000000
923854,2025-08-30 00:00:00.0,NH0550,15.0,30.5,22.75000,1.5,NaN,NaN,NaN,0.000000
